In [115]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import os

This first section is initializing the webdriver object using Selenium. Because the atp website is dynamically loaded, we can't use BeautifulSoup to access the html.

In [116]:
test = 'https://www.atptour.com/en/stats/leaderboard?boardType=return&timeFrame=52Week&surface=all&versusRank=all&formerNo1=false'
browser = webdriver.Firefox(executable_path='/Users/chasemcgill/Downloads/geckodriver')
browser.get(test)


Next, we found the table containing all the information of return leaders in 2023. First we accessed the table element as a whole, then created a list where each element is a separate row.

In [118]:
table = browser.find_element_by_id('leaderboardTable')
r = table.find_elements_by_tag_name('tr')



This is a quick function to make the process of getting each player's information easier. This function parses the text of each row to return a dictionary containing all the pertinent information about a player, like their name and return rating.

In [119]:
def GetInfo(id, row = r):
    t = row[id].text
    first_slash = t.index('\n')
    standing = t[:first_slash]
    name = t[first_slash+1:t.rfind('\n')]
    stats = t[t.rfind('\n')+1:].split(' ')
    
    return {'Name': name, 'Standing': standing, 'Return Rating': float(stats[0]), '% 1st Serve Return Points W'\
            : float(stats[1][:-1]),\
            '% 2nd Serve Return Points W':float(stats[2][:-1]), '% Return Games Won': float(stats[3][:-1]),\
            '% Break Points Converted': float(stats[4][:-1])}

Finally, this cell is where the dataframe containing all the information is created. We used the GetInfo() function to  create the first row of the data frame, and then a for loop to fill it in with all the other player's information.

In [121]:
y = GetInfo(0)
return_df = pd.DataFrame(y, index = [0])
for x in range(len(r))[1:]:
    record = pd.Series(GetInfo(x))
    return_df = pd.concat([return_df, record.to_frame().T], ignore_index = True)

return_df

,Name,Standing,Return Rating,% 1st Serve Return Points W,% 2nd Serve Return Points W,% Return Games Won,% Break Points Converted
0,Daniil Medvedev,1,166.1,33.6,54.2,31.3,47.0
1,Carlos Alcaraz,2,162.0,35.5,53.8,32.2,40.5
2,Jannik Sinner,3,160.8,33.1,54.9,29.9,42.9
3,Novak Djokovic,4,159.1,33.4,54.8,29.2,41.7
4,Alex de Minaur,5,157.7,33.5,52.1,29.3,42.8
...,...,...,...,...,...,...,...
75,Quentin Halys,76,121.0,25.1,44.5,14.8,36.6
76,Brandon Nakashima,77,118.1,27.1,45.7,14.8,30.5
77,Alexei Popyrin,78,117.1,25.7,45.6,14.4,31.4
78,Christopher Eubanks,79,116.6,22.5,45.6,13.0,35.5


This cell is just converting the df to a csv that will appear in the GitHub to make working with it easier.

In [114]:
return_df.to_csv('serve_return.csv')
